https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch17.html#idm46582406873856

reprlib.repr is a utility function to generate abbreviated string representations of data structures that can be very large. By default, reprlib.repr limits the generated string to 30 characters. 

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __getitiem__(self, index):
        return self.words[index]
    
    def __len__(self):
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)

We can call iter() with two arguments to create an iterator from a function or any callable object. In this usage, the first argument must be a callable to be invoked repeatedly (with no arguments) to produce values, and the second argument is a sentinel: a marker value which, when returned by the callable, causes the iterator to raise StopIteration instead of yielding the sentinel. * -- The callable must be one that does not require arguments.

In [1]:
import random

def d6():
    return random.randint(1, 6)

d6_iter = iter(d6, 1)  # this will invoke d6 until the sentinel value is returned

for roll in d6_iter:
    print(roll)

5
4
4
2


We don't see 1 in the results, because it is the sentinel value

### Sidebar that illustrates the difference between iterators and iterables:

In [2]:
L = [1, 2] #iterable
for x in L:
    print('x', x)
    for y in L:
        print('  y', y)

x 1
  y 1
  y 2
x 2
  y 1
  y 2


In [3]:
it = iter([1,2])  #iterator
for x in it:
    print('x', x)
    for y in it:
        print('y', y)

x 1
y 2


The following is the Sentence class built with the Iterator design pattern:

The Sentence class below is iterable because it implements the __iter__ special method, which builds and returns a SentenceIterator. That’s how an iterable and an iterator are related.

Iterators are also iterable, but iterables are not iterators.

It may be tempting to implement __next__ in addition to __iter__ in the Sentence class, making each Sentence instance at the same time an iterable and iterator over itself. But this is rarely a good idea. It’s also a common antipattern:

Use the Iterator pattern:

- to access an aggregate object’s contents without exposing its internal representation.

- to support multiple traversals of aggregate objects.

- to provide a uniform interface for traversing different aggregate structures (that is, to support polymorphic iteration).

##### see better implementations after the cell below

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:

    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        return SentenceIterator(self.words)


class SentenceIterator:

    def __init__(self, words):
        self.words = words
        self.index = 0

    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

#### using a generator instead:

Much more pythonic!!

In [8]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text)
    
    def __iter__(self):
        for word in self.words:
            yield word

In [9]:
s = Sentence("I'm Henry VII; I am.")

print(*s)

I m Henry VII I am


A generator YIELDs values. It does not RETURN values. 

In [4]:
def gen_AB():
    print('start')
    yield 'A'
    print('continue')
    yield 'B'
    print('end.')

for c in gen_AB():
    print('-->', c)
    

start
--> A
continue
--> B
end.


Lazy Sentence Generator:

Previous versions were not lazy because the __iter__ eagerly built the list of all words int he text, and binded them to self.words. This allows use to call next() on our sentence. 
- Building the whole list is in vain if we don't use all the items. 
- If the sentence is huge, then memory gets consumed creating that list.

The loest __iter__ makes the class no longer uses yield, but uses a generator expression, which amounts to the same thing.

Generator expressions are syntactic sugar. My rule of thumb in choosing the syntax to use is simple: if the generator expression spans more than a couple of lines, I prefer to code a generator function for the sake of readability.

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')


class Sentence:

    def __init__(self, text):
        self.text = text

    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text):
            yield match.group

    # OR:
    # def __iter__(self):
    #     return(match.group for match in RE_WORD.find_iter(self.text))

All generators are iterators - and in practice most of the iterators used in python are generators. An iterator implements '__next__'; a generator uses 'yield'. Asynchronous generators are kind of bomb and will be examined later.

itertools.takewhile function: it returns a generator that consumes another generator and stops when a given predicate evaluates to False. So we can combine the two and write this:

In [10]:
import itertools

gen = itertools.takewhile(lambda n: n<3, itertools.count(1, .5))
list(gen)

[1, 1.5, 2.0, 2.5]

Some itertools: https://learning.oreilly.com/library/view/fluent-python-2nd/9781492056348/ch17.html#ap_itertools_sec

A coroutine is really a generator function, created with the yield keyword in its body. And a coroutine object is physically a generator object.

### - Generators produce data for iteration

### - Coroutines are consumers of data

### - To keep your brain from exploding, don’t mix the two concepts together

### - Coroutines are not related to iteration

### Note: There is a use of having `yield` produce a value in a coroutine, but it’s not tied to iteration

In a coroutine, total and count can be local variables: 

#### No instance attributes or closures are needed to keep the context while the coroutine is suspended waiting for the next .send(). 

That’s why coroutines are attractive replacements for callbacks in asynchronous programming—they keep local state between activations.

In [13]:
from collections.abc import Generator

def averager() -> Generator[float, float, None]:
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count
    

In [17]:
coro_avg = averager()

next(coro_avg)

0.0

In [18]:
coro_avg.send(10)

10.0

In [19]:
coro_avg.send(30)

20.0

In [20]:
coro_avg.send(5)

15.0

In [21]:
coro_avg.send(20)

16.25

In [22]:
coro_avg.close()

In [23]:
coro_avg.send(5)

StopIteration: 

1
This function returns a generator that yields float values, accepts float values via .send(), and does not return a useful value.14

2
This infinite loop means the coroutine will keep on yielding averages as long as the client code sends values.

3
The yield statement here suspends the coroutine, yields a result to the client, and—later—gets a value sent by the caller to the coroutine, starting another iteration of the infinite loop.

In a coroutine, total and count can be local variables: no instance attributes or closures are needed to keep the context while the coroutine is suspended waiting for the next .send(). That’s why coroutines are attractive replacements for callbacks in asynchronous programming—they keep local state between activations.